In [1]:
#todo:normalize, take in regard pid, as of now i assume that the order is the same in features and labels


In [2]:
from sklearn.linear_model import Ridge
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
from tensorflow.keras import layers
import sklearn


In [3]:
tfeaturespath = 'task2_k49am2lqi/train_features.csv'
testfeaturespath = 'task2_k49am2lqi/test_features.csv'
tlabelspath = 'task2_k49am2lqi/train_labels.csv'
tfeatures = pd.read_csv(tfeaturespath)
testfeatures = pd.read_csv(testfeaturespath)
tlabels = pd.read_csv(tlabelspath)

#feature construction
raw = np.zeros((tfeatures.shape[0]//12, (tfeatures.loc[1].size-2)*3-2))
for i in tqdm(range(raw.shape[0])):
    raw[i] = tfeatures[12*i:12*(i+1)].describe().loc[['mean', 'min', 'max']].unstack().drop(['pid', 'Time', ('Age','min'), ('Age', 'max')])
np.save('task2_k49am2lqi/raw_with_nan', raw)
#impute nans
mean = np.nanmean(raw, axis = 0)
inds = np.where(np.isnan(raw))
raw[inds] = np.take(mean, inds[1])
np.save('task2_k49am2lqi/raw', raw)

#feature construction testset
testraw = np.zeros((testfeatures.shape[0]//12, (testfeatures.loc[1].size-2)*3-2))
for i in tqdm(range(testraw.shape[0])):
    testraw[i] = testfeatures[12*i:12*(i+1)].describe().loc[['mean', 'min', 'max']].unstack().drop(['pid', 'Time', ('Age','min'), ('Age', 'max')])
np.save('task2_k49am2lqi/testraw_with_nan', testraw)
#impute nans
mean = np.nanmean(testraw, axis = 0)
inds = np.where(np.isnan(testraw))
testraw[inds] = np.take(mean, inds[1])
np.save('task2_k49am2lqi/testraw', testraw)

In [4]:
raw = np.load('task2_k49am2lqi/raw.npy')
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(raw)
raw = scaler.transform(raw)

FileNotFoundError: [Errno 2] No such file or directory: 'task2_k49am2lqi/raw.npy'

In [ ]:
tlabels1 = tlabels[['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']].values
tlabels2 = tlabels['LABEL_Sepsis'].values
tlabels3 = tlabels[['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']].values

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(raw, tlabels1, test_size=0.2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(raw, tlabels2, test_size=0.2)
X_train3, X_test3, y_train3, y_test3 = train_test_split(raw, tlabels3, test_size=0.2)

In [ ]:
y_train33 = [y_train3[:,0], y_train3[:,1], y_train3[:,2], y_train3[:,3]]
y_test33 = [y_test3[:,0], y_test3[:,1], y_test3[:,2], y_test3[:,3]]

In [ ]:
model1 = keras.Sequential(
    [
        keras.Input(shape=(raw[0].size,)),
        layers.Dense(50, activation="relu", name="layer1"),
        layers.Dense(30, activation="relu", name="layer2"),
        layers.Dense(25, activation="relu", name="layer4"),
        layers.Dense(tlabels1.shape[1], activation='sigmoid', name="layer5")
    ]
)

model2 = keras.Sequential(
    [
        keras.Input(shape=(raw[0].size,)),
        layers.Dense(50, activation="relu", name="layer1"),
        layers.Dense(30, activation="relu", name="layer2"),
        layers.Dense(25, activation="relu", name="layer4"),
        layers.Dense(1, activation='sigmoid', name="layer5")
    ]
)


input3 = keras.Input(shape=(raw[0].size,))
x1 = layers.Dense(50, activation="relu")(input3)
x2 = layers.Dense(50, activation="relu")(input3)
x3 = layers.Dense(50, activation="relu")(input3)
x4 = layers.Dense(50, activation="relu")(input3)
        
y1 = layers.Dense(30, activation="relu")(x1)
y2 = layers.Dense(30, activation="relu")(x2)
y3 = layers.Dense(30, activation="relu")(x3)
y4 = layers.Dense(30, activation="relu")(x4)

z1 = layers.Dense(10, activation="relu")(y1)
z2 = layers.Dense(10, activation="relu")(y2)
z3 = layers.Dense(10, activation="relu")(y3)
z4 = layers.Dense(10, activation="relu")(y4)

out1 = layers.Dense(1, activation="linear")(y1)
out2 = layers.Dense(1, activation="linear")(y2)
out3 = layers.Dense(1, activation="linear")(y3)
out4 = layers.Dense(1, activation="linear")(y4)

model3 = keras.Model(inputs=input3, outputs=[out1,out2,out3,out4])

In [ ]:
def coeff_determination(y_true, y_pred):
    from tensorflow.keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
model1.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer='adam',metrics=[tf.keras.metrics.AUC(curve = 'ROC')])
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=[tf.keras.metrics.AUC(curve = 'ROC')])
model3.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam',metrics=[coeff_determination])

In [ ]:
history1 = model1.fit(
    X_train1,
    y_train1,
    batch_size=64,
    epochs=5,
    validation_data=(X_test1, y_test1),
)
model1.save('task2_k49am2lqi/model1')

In [ ]:
history2 = model2.fit(
    X_train2,
    y_train2,
    batch_size=64,
    epochs=5,
    validation_data=(X_test2, y_test2),
)
model2.save('task2_k49am2lqi/model2')

In [ ]:
history3 = model3.fit(
    X_train3,
    y_train33,
    batch_size=64,
    epochs=5,
    validation_data=(X_test3, y_test33),
)
model3.save('task2_k49am2lqi/model3')

In [ ]:
testraw = np.load('task2_k49am2lqi/testraw.npy')
testraw = scaler.transform(testraw)

In [ ]:
result1 = model1.predict(testraw)
result2 = model2.predict(testraw)
result3 = model3.predict(testraw)

In [ ]:
naming = ['pid', 'LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2','LABEL_Sepsis','LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
upload = np.zeros((testraw.shape[0], len(naming)))

In [ ]:
pid = testfeatures['pid'].values
PID = np.empty(testraw.shape[0])
for i in range(PID.size):
    PID[i] = pid[12*i]

In [ ]:
upload[:,0] = PID
upload[:,1:11] = result1
upload[:,11] = result2.ravel()
upload[:,12] = result3[0].ravel()
upload[:,13] = result3[1].ravel()
upload[:,14] = result3[2].ravel()
upload[:,15] = result3[3].ravel()

In [ ]:
df =pd.DataFrame(data=upload, columns = naming)

In [ ]:
df.to_csv('prediction4.zip', index=False, float_format='%.3f', compression = 'zip')